In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import os, sys
sys.path.append(os.getcwd())

import time
import functools
import argparse

import numpy as np

from tensorboardX import SummaryWriter

import pdb

from models.wgan import *

import torch
import torchvision
from torch import nn
from torch import autograd
from torch import optim
from torchvision import transforms, datasets
from torch.autograd import grad
from timeit import default_timer as timer
from tqdm import tqdm

import torch.nn.init as init
import matplotlib.pyplot as plt

In [ ]:
# lsun lmdb data set can be download via https://github.com/fyu/lsun
# 64x64 ImageNet at http://image-net.org/small/download.php
DATA_DIR = '/data/animefaces' # Replace your image data path here
VAL_DIR = '/data/animefaces'
IMAGE_DATA_SET = 'animefaces' 
# change this to something else, e.g. 'imagenets' or 'raw' if your data is just a folder of raw images. 
# Example: 
# IMAGE_DATA_SET = 'raw'
# If you use lmdb, you'll need to write the loader by yourself. Please check load_data function

TRAINING_CLASS = [] # IGNORE this if you are NOT training on lsun, or if you want to train on other classes of lsun, then change it accordingly
VAL_CLASS = [] # IGNORE this if you are NOT training on lsun, or if you want to train on other classes of lsun, then change it accordingly

if len(DATA_DIR) == 0:
    raise Exception('Please specify path to data directory in gan_64x64.py!')

RESTORE_MODE = True # if True, it will load saved model from OUT_PATH and continue to train
START_ITER = 0 # starting iteration 
OUTPUT_PATH = '/content/drive/My Drive/data/models/animefaces/' # output path where result (.e.g drawing images, cost, chart) will be stored
# MODE = 'wgan-gp'
DIM = 64 # Model dimensionality
CRITIC_ITERS = 5 # How many iterations to train the critic for
GENER_ITERS = 1
N_GPUS = 1 # Number of GPUs
BATCH_SIZE = 64# Batch size. Must be a multiple of N_GPUS
END_ITER = 100000 # How many iterations to train for
LAMBDA = 10 # Gradient penalty lambda hyperparameter
OUTPUT_DIM = 64*64*3 # Number of pixels in each image

In [ ]:
def weights_init(m):
    if isinstance(m, MyConvo2d): 
        if m.conv.weight is not None:
            if m.he_init:
                init.kaiming_uniform_(m.conv.weight)
            else:
                init.xavier_uniform_(m.conv.weight)
        if m.conv.bias is not None:
            init.constant_(m.conv.bias, 0.0)
    if isinstance(m, nn.Linear):
        if m.weight is not None:
            init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0.0)

def load_data(path_to_folder, classes):
    data_transform = transforms.Compose([
                 transforms.Resize(64),
                 transforms.CenterCrop(64),
                 transforms.ToTensor(),
                 transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])
                ])
    if IMAGE_DATA_SET == 'lsun':
        dataset =  datasets.LSUN(path_to_folder, classes=classes, transform=data_transform)
    else:
        dataset = datasets.ImageFolder(root=path_to_folder,transform=data_transform)
    dataset_loader = torch.utils.data.DataLoader(dataset,batch_size=BATCH_SIZE, shuffle=True, num_workers=5, drop_last=True, pin_memory=True)
    return dataset_loader

def training_data_loader():
    return load_data(DATA_DIR, TRAINING_CLASS) 

def val_data_loader():
    return load_data(VAL_DIR, VAL_CLASS) 

def calc_gradient_penalty(netD, real_data, fake_data):
    alpha = torch.rand(BATCH_SIZE, 1)
    alpha = alpha.expand(BATCH_SIZE, int(real_data.nelement()/BATCH_SIZE)).contiguous()
    alpha = alpha.view(BATCH_SIZE, 3, DIM, DIM)
    alpha = alpha.to(device)
    
    fake_data = fake_data.view(BATCH_SIZE, 3, DIM, DIM)
    interpolates = alpha * real_data.detach() + ((1 - alpha) * fake_data.detach())

    interpolates = interpolates.to(device)
    interpolates.requires_grad_(True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).to(device),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradients = gradients.view(gradients.size(0), -1)                              
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty

def generate_image(netG, noise=None):
    if noise is None:
        noise = gen_rand_noise()

    with torch.no_grad():
    	noisev = noise 
    samples = netG(noisev)
    samples = samples.view(BATCH_SIZE, 3, 64, 64)
    samples = samples * 0.5 + 0.5
    return samples

def gen_rand_noise():
    noise = torch.randn(BATCH_SIZE, 128)
    noise = noise.to(device)

    return noise

cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")
fixed_noise = gen_rand_noise() 

if RESTORE_MODE:
    aG = torch.load(OUTPUT_PATH + "generator.pt")
    aD = torch.load(OUTPUT_PATH + "discriminator.pt")
else:
    aG = GoodGenerator(64,64*64*3)
    aD = GoodDiscriminator(64)
    
    aG.apply(weights_init)
    aD.apply(weights_init)

In [ ]:
LR = 1e-4
optimizer_g = torch.optim.Adam(aG.parameters(), lr=LR, betas=(0,0.9))
optimizer_d = torch.optim.Adam(aD.parameters(), lr=LR, betas=(0,0.9))
one = torch.FloatTensor([1])
mone = one * -1
aG = aG.to(device)
aD = aD.to(device)
one = one.to(device)
mone = mone.to(device)
writer = SummaryWriter(OUTPUT_PATH + 'runs/exp-1')

In [ ]:
fixed_noise = gen_rand_noise() 

In [ ]:
# Lists to keep track of progress
ncritic = 5
current_iter = 0

In [ ]:
def train():
    global current_iter
    dataloader = training_data_loader() 
    dataiter = iter(dataloader)
    with tqdm(initial=current_iter,total=END_ITER) as pbar:
      for iteration in range(current_iter, END_ITER):
          #---------------------TRAIN G------------------------
          for p in aD.parameters():
              p.requires_grad_(False)  # freeze D

          gen_cost = None
          for i in range(GENER_ITERS):
              aG.zero_grad()
              noise = gen_rand_noise()
              noise.requires_grad_(True)
              fake_data = aG(noise)
              gen_cost = aD(fake_data)
              gen_cost = gen_cost.mean()
              gen_cost.backward(mone)
              gen_cost = -gen_cost

          optimizer_g.step()
          #---------------------TRAIN D------------------------
          for p in aD.parameters():  # reset requires_grad
              p.requires_grad_(True)  # they are set to False below in training G
          for i in range(CRITIC_ITERS):

              aD.zero_grad()

              # gen fake data and load real data
              noise = gen_rand_noise()
              with torch.no_grad():
                  noisev = noise  # totally freeze G, training D
              fake_data = aG(noisev).detach()
              batch = next(dataiter, None)
              if batch is None:
                  dataiter = iter(dataloader)
                  batch = dataiter.next()
              batch = batch[0] #batch[1] contains labels
              real_data = batch.to(device) #TODO: modify load_data for each loading

              # train with real data
              disc_real = aD(real_data)
              disc_real = disc_real.mean()

              # train with fake data
              disc_fake = aD(fake_data)
              disc_fake = disc_fake.mean()

              # train with interpolates data
              gradient_penalty = calc_gradient_penalty(aD, real_data, fake_data)

              # final disc cost
              disc_cost = disc_fake - disc_real + gradient_penalty
              disc_cost.backward()
              w_dist = disc_fake  - disc_real
              optimizer_d.step()
              #------------------VISUALIZATION----------
              if i == CRITIC_ITERS-1:
                  writer.add_scalar('runs/disc_cost', disc_cost, iteration)
                  writer.add_scalar('data/disc_fake', disc_fake, iteration)
                  writer.add_scalar('data/disc_real', disc_real, iteration)
                  writer.add_scalar('runs/gradient_pen', gradient_penalty, iteration)
                  #writer.add_scalar('data/d_conv_weight_mean', [i for i in aD.children()][0].conv.weight.data.clone().mean(), iteration)
                  #writer.add_scalar('data/d_linear_weight_mean', [i for i in aD.children()][-1].weight.data.clone().mean(), iteration)
                  #writer.add_scalar('data/fake_data_mean', fake_data.mean(), iteration)
                  #writer.add_scalar('data/real_data_mean', real_data.mean(), iteration)
                  #if iteration %200==99:
                  #    paramsD = aD.named_parameters()
                  #    for name, pD in paramsD:
                  #      writer.add_histogram("D." + name, pD.clone().data.cpu().numpy(), iteration)

          #---------------VISUALIZATION---------------------
          writer.add_scalar('runs/gen_cost', gen_cost, iteration)
          writer.add_scalar('runs/wasserstein_distance', w_dist, iteration)

          if iteration % 200 == 199:
              gen_images = generate_image(aG, fixed_noise)
              torchvision.utils.save_image(gen_images, OUTPUT_PATH + 'images/samples_{}.png'.format(iteration), nrow=8, padding=2)
              grid_images = torchvision.utils.make_grid(gen_images, nrow=8, padding=2)
              writer.add_image('runs/images', grid_images, iteration)
    #----------------------Save model----------------------
              torch.save(aG, OUTPUT_PATH + "generator.pt")
              torch.save(aD, OUTPUT_PATH + "discriminator.pt")
              torch.save(aG.state_dict(), OUTPUT_PATH + "generator_state.pt")
              torch.save(aD.state_dict(), OUTPUT_PATH + "discriminator_state.pt")
          pbar.update()
          current_iter += 1

In [ ]:
train()

In [ ]:
def plot_batch(title, image_list):
    plt.figure(figsize=(8,8))
    plt.subplot(1,1,1)
    plt.axis("off")
    plt.title(title)
    plt.imshow(image_list)
    plt.show()

In [ ]:
def compare_real_fake():
    # Grab a batch of real images from the dataloader
    real_batch = next(iter(training_data_loader()))
    real_images = torchvision.utils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu()
    fake_images = torchvision.utils.make_grid(generate_image(), nrow=8, padding=2)

    # Plot the real images
    plot_batch("Real Images", real_images)

    # Plot the fake images from the last epoch
    plot_batch("Fake Images", fake_images)

**Real Images vs. Fake Images**

Finally, lets take a look at some real images and fake images side by
side.




In [ ]:
compare_real_fake()